In [1]:
# 7/2/21: https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2
# end cite

In [2]:

import numpy as np
import os
import pandas as pd

from os.path import join, exists

import config
np.random.seed(config.SEED)

In [3]:
final_save_path = join(config.prov_csv_dir, 'pvd_utt_glosses_phono_cleaned_inflated_to_next_notebook.pkl')

if config.regenerate:
    print('For re-generating final all_tokens_phono before splits, please see other notebook. Using cached values.')

all_tokens_phono = pd.read_pickle(final_save_path)

For re-generating final all_tokens_phono before splits, please see other notebook. Using cached values.


In [4]:
from utils import split_gen, sampling, data_cleaning, load_models, data_cleaning, transformers_bert_completions
from utils_child import child_split_gen, child_models

## Get the samples and splits for age/all splits

In [5]:

# Do this for each of success and yyy, then merge them together.

all_tokens_phono_valid = data_cleaning.find_transcripts_with_successes_and_yyy(all_tokens_phono)

# Split train/val/test: 25/50/50.

split_attr = 'transcript_id'

phono_train_val_idxs, phono_eval_idxs = split_gen.determine_split_idxs(all_tokens_phono_valid, split_attr, 0.5)

phono_train_val = all_tokens_phono_valid[all_tokens_phono_valid.transcript_id.isin(phono_train_val_idxs)]
phono_train_idxs, phono_val_idxs = split_gen.determine_split_idxs(phono_train_val, split_attr, 0.5)

for phase, idx_set in zip(['train', 'val', 'eval'], [phono_train_idxs, phono_val_idxs, phono_eval_idxs]):

    # It's on transcript_id, not actual idx, so this is OK.
    # all_tokens_phono will receive the val/eval phase marking where it applies.

    this_phase_data, all_tokens_phono = split_gen.assign_and_find_phase_data(phase, split_attr, idx_set, all_tokens_phono, 'phase_sample')

all_tokens_phono = data_cleaning.augment_target_child_year(all_tokens_phono)

# Below: For debugging only
all_tokens_phono.to_pickle(join(config.prov_dir, 'pvd_all_tokens_phono_for_eval_before_child.pkl')) 


In [6]:

young_phono, old_phono = split_gen.get_age_split_data(all_tokens_phono)

phono_pool = [
    all_tokens_phono,
    young_phono,
    old_phono,
]

model_args = [('all', 'all'), ('age', 'young'), ('age', 'old')]

for (split_name, dataset_name), this_phono_raw in zip(model_args, phono_pool):
    
    print('Processing', split_name, dataset_name)
    phono_phase = this_phono_raw[(this_phono_raw.phase_sample == 'val') & (this_phono_raw.partition == 'success')]

    # age = None means don't filter on a given age
    result_beta_sample = sampling.sample_successes('beta', split_name, dataset_name, None, phono_phase, 'val')        

    print('\tbeta sample', result_beta_sample.shape)


# Handle sparse ages later in the pipeline, if any

used_ages = data_cleaning.get_years(all_tokens_phono)

for age in used_ages:
    
    for phase in ['val', 'eval']:
        
        for sample_func, sample_name in zip([sampling.sample_successes, sampling.sample_yyy], ['success', 'yyy']):

            print(f'for {sample_name}')

            phono_phase = all_tokens_phono[(all_tokens_phono.phase_sample == phase) & (all_tokens_phono.partition == sample_name)]
            this_age_sample = sample_func('models_across_time', None, None, age, phono_phase, phase)        

            print('\tage sample', this_age_sample.shape)

Processing all all
Resampling for: task: beta, split: all, dataset: all, age: None, phase: val
	beta sample (5000, 1)
Processing age young
Resampling for: task: beta, split: age, dataset: young, age: None, phase: val
	beta sample (5000, 1)
Processing age old
Resampling for: task: beta, split: age, dataset: old, age: None, phase: val
	beta sample (5000, 1)
for success
Resampling for: task: models_across_time, split: None, dataset: None, age: 0.5, phase: val
	age sample (1, 1)
for yyy
Resampling for: task: models_across_time, split: None, dataset: None, age: 0.5, phase: val
	age sample (1, 1)
for success
Resampling for: task: models_across_time, split: None, dataset: None, age: 0.5, phase: eval
	age sample (97, 1)
for yyy
Resampling for: task: models_across_time, split: None, dataset: None, age: 0.5, phase: eval
	age sample (101, 1)
for success
Resampling for: task: models_across_time, split: None, dataset: None, age: 1.0, phase: val
	age sample (2316, 1)
for yyy
Resampling for: task: mo

# Child work

In [7]:

split_attr = 'transcript_id'

# 7/25/21: https://www.kite.com/python/answers/how-to-create-an-empty-column-in-a-pandas-dataframe-in-python
all_tokens_phono['phase_child_sample'] = np.nan
all_tokens_phono['phase_child_finetune'] = np.nan
# end cite

for name in sorted(list(set(all_tokens_phono.target_child_name))):
    
    this_partition_folder = split_gen.get_split_folder('child', name, config.finetune_dir)
    
    ## -------- Restricted sampling section
    
    print(f'Processing: {name}')
    
    this_child_phono = all_tokens_phono[(all_tokens_phono.target_child_name == name)]
    
    this_valid_phono = data_cleaning.find_transcripts_with_successes_and_yyy(this_child_phono)
 
    # Sample across ages
    
    complete_phase_idxs = child_split_gen.find_splits_across_ages(this_valid_phono)
        
    for phase_name, idx_set in complete_phase_idxs.items():
        
        # Make a new attribute for all_tokens_phono parallel to phase (which is the val/eval split defined above)
        _, all_tokens_phono = split_gen.assign_and_find_phase_data(phase_name, split_attr, idx_set, all_tokens_phono, phase_label = 'phase_child_sample')
    
    # Beta samples
    
    val_success_pool = all_tokens_phono[
        (all_tokens_phono.partition == 'success')
        & (all_tokens_phono.target_child_name == name)
        & (all_tokens_phono.phase_child_sample == 'val')
    ]
    
    # Note: get_beta_idxs does NOT internally filter things.
    # It's necessary to pass all_tokens_phono-based filtering because all_tokens_phono has the phase information
    # associated with it.
    
    val_sample = child_split_gen.get_beta_idxs(val_success_pool, 'transcript_id')
    
    this_path = sampling.get_sample_path('success', 'beta', 'child', name, eval_phase = 'val')
    val_sample.to_csv(this_path)

    print(f'\tWriting beta samples for phase {phase}, to {this_path}, sample size: {val_sample.shape}, pool size: {len(set(val_success_pool.utterance_id))}')
    
    ## -------- Unrestricted sampling section
    
    # Identify everything that isn't in the sample.
    
    complete_sample_idxs = np.concatenate([complete_phase_idxs[phase] for phase in ['train', 'val', 'eval']])
    
    # Checked that all_tokens_phono is unfiltered pool of information.
    
    train_val_finetune_phono_new = this_child_phono[~this_child_phono.transcript_id.isin(complete_sample_idxs)]

    train_val_finetune_phono_new = data_cleaning.drop_errors(train_val_finetune_phono_new)
    # prepped glosses already done above in Pvd logic
    
    train_merge_out_pool = {}
    
    avail_train_val_finetune_ids = set(train_val_finetune_phono_new.transcript_id)
    num_train_val_finetune_new_ids = len(avail_train_val_finetune_ids)
    
    if num_train_val_finetune_new_ids >= 2:
        train_merge_out_pool['train'], train_merge_out_pool['val'] = split_gen.determine_split_idxs(train_val_finetune_phono_new, 'transcript_id', val_ratio = config.val_ratio)
    elif num_train_val_finetune_new_ids == 1:
        # Prioritize validation because train will receive a larger merge from the previous data.
        train_merge_out_pool['train'], train_merge_out_pool['val'] = np.array([]), np.array(list(avail_train_val_finetune_ids))
    else:
        train_merge_out_pool['train'], train_merge_out_pool['val'] = np.array([]), np.array([])
    
    # Complete_phase_idxs still has some yyy in it.
    # Isolate the parts of train_sample that can be merged with the finetune train phase.
    
    no_errors_phono = data_cleaning.drop_errors(this_child_phono)
    
    train_merge_in_pool = {}
    for phase in ['train', 'val']:
        train_merge_in_pool[phase] = np.unique(no_errors_phono[no_errors_phono.transcript_id.isin(complete_phase_idxs[phase])].transcript_id)
    
    finetune_idxs = {}
    for phase in ['train', 'val']:
        finetune_idxs[phase] = np.concatenate([train_merge_out_pool[phase], train_merge_in_pool[phase]])
    
    ## Identify and write the finetune phases relative to partition information.
    
    for finetune_phase, finetune_idx_set in finetune_idxs.items():
        
        split_gen.write_data_partitions_text(this_child_phono, this_partition_folder, finetune_phase, finetune_idx_set, 'transcript_id', 'phase_child_finetune')
        
        # Re-assign phase information all_tokens_phono
        # because need to limit to this_child_phono above for the writing.
        
        _, all_tokens_phono = split_gen.assign_and_find_phase_data(finetune_phase, split_attr, finetune_idx_set, all_tokens_phono, phase_label = 'phase_child_finetune')
        
        

KeyError: "None of [Index(['Alex', 'Alex', 'Alex', 'Alex', 'Alex', 'Alex', 'Alex', 'Alex', 'Alex',\n       'Alex',\n       ...\n       'William', 'William', 'William', 'William', 'William', 'William',\n       'William', 'William', 'William', 'William'],\n      dtype='object', length=2991865)] are in the [columns]"

In [ ]:
# Write final all_tokens_phono with all split information to the proper place.

if not exists(config.prov_dir):
    os.makedirs(config.prov_dir)
    
all_tokens_phono.to_pickle(join(config.prov_dir, 'pvd_all_tokens_phono_for_eval.pkl'))